In [16]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [17]:
from datasets import load_dataset
ds = load_dataset("llm-wizard/alpaca-gpt4-data-zh", cache_dir="./data",split = "train")

In [18]:
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 48818
})

In [19]:
ds[5]

{'instruction': '辨认不同之处。 ',
 'input': '输入：Twitter，Instagram，Telegram。',
 'output': 'Twitter，Instagram 和 Telegram 都是流行的社交媒体平台，但它们之间也有一些不同之处：\n\n- Twitter: 是一个微博社交媒体平台，用户可以发布 280 字符以内的消息（即推文）并与其他用户互动。它主要关注文字交流，尽管用户也可以分享照片和视频。\n- Instagram: 是一个图片和视频分享的社交媒体平台，用户可以在 Instagram 上发布照片和视频并与其他用户互动。它主要关注图片和视频的分享，尽管用户也可以添加文字说明。\n- Telegram: 是一个即时通讯和社交媒体平台，用户可以通过文本、语音、图片和视频与其他用户即时交流。与 Twitter 和 Instagram 不同，Telegram 更侧重于即时通讯，它还提供了端到端加密的私人聊天功能。'}

In [20]:
ds = ds.train_test_split(test_size=0.8)
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 9763
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 39055
    })
})

In [21]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2TokenizerFast(name_or_path='Qwen/Qwen1.5-0.5B', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [23]:
tokenized_ds = ds.map(process_func, remove_columns=ds["train"].column_names)
tokenized_ds

Map:   0%|          | 0/39055 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 9763
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 39055
    })
})

In [9]:
tokenizer.decode(tokenized_ds["train"][1]["input_ids"])

'Human: 创建完成项目所需任务的时间表。 \n输入：这是一个构建移动应用原型的项目。\n\nAssistant: 为了创建完成项目所需的时间表，你需要提供更多的信息，如项目的预期日期，每项任务所需的时间，以及每天可用的工作小时数等。如果提供这些信息，我可以对项目的计划进一步提供帮助。<|endoftext|>'

In [10]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds["train"][1]["labels"])))

'为了创建完成项目所需的时间表，你需要提供更多的信息，如项目的预期日期，每项任务所需的时间，以及每天可用的工作小时数等。如果提供这些信息，我可以对项目的计划进一步提供帮助。<|endoftext|>'

In [11]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [24]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=100,
    num_train_epochs=1,
    report_to=['tensorboard'],
)

In [25]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"], 
    eval_dataset=tokenized_ds["test"],     
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [26]:
trainer.train()

Step,Training Loss
100,1.949100
200,1.888200
300,1.817400


TrainOutput(global_step=305, training_loss=1.8848678401259125, metrics={'train_runtime': 1168.1448, 'train_samples_per_second': 8.358, 'train_steps_per_second': 0.261, 'total_flos': 4123982640046080.0, 'train_loss': 1.8848678401259125, 'epoch': 1.0})

In [27]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [28]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=2048) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 当进行考试时，你可以运用以下几种技巧来提高你的成绩：\n\n1. 准备充分：考试前复习材料和考试内容，做好复习计划，有助于你更好地应对考试。\n\n2. 认真审题：仔细审题是考试中最重要的一部分，能有效避免因粗心而丢分的情况。\n\n3. 组织好自己的时间和分心：考试时分散注意力会导致错误率增加，因此要集中精力，确保按时间表进行考试。\n\n4. 注意语调：在考场上，良好的语调会使你更容易回答问题，避免由于语音问题而丢分。\n\n5. 努力尝试：如果你能够持之以恒，发挥出你的水平，那么你将有更多的可能获得好的成绩。\n\n6. 磋商心态：考试不是一场游戏，而是为了准备和表现的。保持一个积极、乐观的态度，将有助于你取得好的成绩。\n\n7. 反思总结：考试结束之后，认真分析自己的成绩，并根据考试中所出现的问题进行改进和提高，将有助于你更好地为下一次考试做好准备。'}]

In [31]:
re = pipe(ipt, max_length=256, do_sample=True, )[0]["generated_text"]
re

Both `max_new_tokens` (=2048) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'Human: 考试有哪些技巧？\n\nAssistant: 考试的过程是一项具有挑战性的任务，以下是一些可以帮助你有效迎接考试的技巧:\n\n1. 事先充分准备: 定期复习考试的重要内容和知识点。\n2. 分析考题: 找出题型和知识点，并按照考试要求进行总结和复习。\n3. 做习题解题: 参开一些习题和解答题，对自己学习能力和判断能力进行测试。\n4. 让他人帮忙: 请家人，朋友或老师来帮你练习解题技巧，帮助你更好地应对考试。\n5. 重视课堂: 在课堂上积极回答问题，与老师或同学互动，可以帮助你更快地消化考题。\n6. 注意饮食和锻炼: 注意保持良好的生活习惯，保证营养均衡，身体健康，有助于考试顺利进行。\n7. 调用自己的情绪: 在考试中遇到困难时，可以通过冥想或者深呼吸来提升的情绪水平，保持冷静和集中精神。\n\n希望以上提示能够给你带来帮助，祝你考试顺利，取得优异的成绩！'

In [32]:
print(re)

Human: 考试有哪些技巧？

Assistant: 考试的过程是一项具有挑战性的任务，以下是一些可以帮助你有效迎接考试的技巧:

1. 事先充分准备: 定期复习考试的重要内容和知识点。
2. 分析考题: 找出题型和知识点，并按照考试要求进行总结和复习。
3. 做习题解题: 参开一些习题和解答题，对自己学习能力和判断能力进行测试。
4. 让他人帮忙: 请家人，朋友或老师来帮你练习解题技巧，帮助你更好地应对考试。
5. 重视课堂: 在课堂上积极回答问题，与老师或同学互动，可以帮助你更快地消化考题。
6. 注意饮食和锻炼: 注意保持良好的生活习惯，保证营养均衡，身体健康，有助于考试顺利进行。
7. 调用自己的情绪: 在考试中遇到困难时，可以通过冥想或者深呼吸来提升的情绪水平，保持冷静和集中精神。

希望以上提示能够给你带来帮助，祝你考试顺利，取得优异的成绩！
